In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [2]:
def create_layer(n_in, n_out, act_func = None):
    layer = {}
    layer["weights"] = np.random.random((n_out, n_in + 1))
    if act_func:
        layer["act_func"] = act_func()
    else:
        layer["act_func"] = None
    return layer

In [3]:
def forwardpass_layer(layer, X):
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1)
    if layer["act_func"]:
        temp = torch.tensor((X @ layer["weights"].T).astype(np.float32))
        return layer["act_func"](temp).numpy()
    else:
        return X @ layer["weights"].T

In [4]:
def create_network(layers, act_funcs = None):
    network = {}
    network["layers"] = []
    network["layer_sizes"] = []
    
    if act_funcs is None:
        for i in range(len(layers) - 1):
            network["layers"] += [create_layer(layers[i], layers[i + 1])]
            network["layer_sizes"] += [(layers[i] + 1, layers[i + 1])]
            
    else:
        for i in range(len(layers) - 1):
            network["layers"] += [create_layer(layers[i], layers[i + 1], act_funcs[i])]
            network["layer_sizes"] += [(layers[i] + 1, layers[i + 1])]
            
    return network

In [5]:
def forwardpass_network(network, X):
    for layer in network["layers"]:
        X = forwardpass_layer(layer, X)
    return X

In [6]:
def get_weight_vector(network):
    weight_vector = []
    
    for layer in network["layers"]:
        weight_vector += [layer["weights"].flatten()]
        
    weight_vector = np.concatenate(weight_vector)
    return weight_vector

In [7]:
def set_layer_weights(network, weight_vector):
    start = 0
    
    for l in range(len(network["layer_sizes"])):
        input_size, output_size = network["layer_sizes"][l]
        weights = weight_vector[start:start + input_size*output_size]
        weights = weights.reshape((output_size, input_size))
        network["layers"][l]["weights"] = weights
        start += input_size * output_size

In [8]:
def create_belief(ind_size, startMean, startStd):
    belief = {}
    belief["ind_size"] = ind_size
    
    try:
        _ = len(startMean)
        belief["mean"] = np.full(ind_size, startMean)
        belief["std"] = np.full(ind_size, startStd)
        
    except:
        belief["mean"] = startMean
        belief["std"] = startStd
        
    return belief

In [9]:
def bel_generate_population(belief, pop_size):
    belief["pop"] = (belief["mean"] + belief["std"] * np.random.randn(pop_size, belief["ind_size"]))

In [10]:
def update_traits(belief):
    temp = belief["pop"].T
    for i in range(len(temp)):
        belief["mean"] = np.mean(temp[i])
        belief["std"] = np.std(temp[i])

In [11]:
def create_ca(model, X, Y, beliefs, mut_rate = 1):
    ca = {}
    ca["model"] = model
    ca["X"] = X
    ca["Y"] = Y
    ca["mut_rate"] = mut_rate
    
    ca["ind_size"] = get_weight_vector(model).shape[0]
    ca["cultures"] = []
    
    for belief_params in beliefs:
        ca["cultures"] += [create_belief(ca["ind_size"], belief_params[0], belief_params[1])]
        
    return ca

In [12]:
def generate_population(ca, pop_sizePerBelief):
    ca["pop_sizePerBelief"] = pop_sizePerBelief
    ca["pop_size"] = len(ca['cultures']) * pop_sizePerBelief
    for belief in ca["cultures"]:
        bel_generate_population(belief, pop_sizePerBelief)
        try:
            ca["pop"] = np.concatenate((ca["pop"], belief["pop"]))
        except:
            ca["pop"] = belief["pop"]

In [13]:
def select_next_population_roulette(ca):
    evaluate_population(ca)
    temp = ca["pop"].copy()
    
    for i in range(ca["pop_size"]):
        ca["scores"] = ca["scores"] / np.sum(ca["scores"])

        roulette = np.cumsum(ca["scores"])

        newPopIndex = []
        for count in range(ca["pop_size"]):
            roulette_ball = np.random.random()
            newPopIndex += [(roulette >= roulette_ball).tolist().index(True)]
                
    ca["pop"] = temp[newPopIndex]

In [15]:
def select_next_population(ca):
    evaluate_population(ca)
    
    nextPopIndices = np.argsort(ca["scores"])[::-1][:ca["pop_size"]]
    
    ca["pop"] = ca["pop"][nextPopIndices]   

In [16]:
def mutate(ca, ind):
    if np.random.random() > (1-ca["mut_rate"]):
        index = np.random.randint(ca["ind_size"])
        ind[index] = np.random.randint(100) * np.random.rand()
        
    return ind

In [17]:
def cross_over(ca, p1, p2):
    point = np.random.randint(ca["ind_size"])
    
    off1 = np.concatenate((p1[:point], p2[point:]))
    off2 = np.concatenate((p1[point:], p2[:point]))
    
    off1 = mutate(ca, off1)
    off2 = mutate(ca, off2)
    
    return np.array([off1, off2])

In [18]:
def reproduce(ca):
    for p1 in range(ca["pop_size"]):
        for p2 in range(ca["pop_size"]):
            if p1 != p2:
                ca["pop"] = np.concatenate((ca["pop"], cross_over(ca, ca["pop"][p1], ca["pop"][p2])))
            

In [19]:
def evaluate_population(ca):
    scores = []
    for individual in ca["pop"]:
        set_layer_weights(ca["model"], individual)
        pred = np.round(forwardpass_network(ca["model"], ca["X"]))
        scores += [accuracy_score(ca["Y"], pred)]            
    ca["scores"] = np.array(scores)

In [20]:
def classify(ca):
    for belief in ca["cultures"]:
        z = np.sum((ca["pop"] - belief["mean"]), axis = 1) / (belief["std"])
        newPopulationOfBelief = np.argsort(z)[:ca["pop_sizePerBelief"]]
        belief["pop"] = ca["pop"][newPopulationOfBelief]
        ca["pop"] = ca["pop"][np.argsort(z)[ca["pop_sizePerBelief"]:]]
        
        try:
            temp = np.concatenate((belief["pop"], temp))
        except:
            temp = belief["pop"]
            
    for belief in ca["cultures"]:
        ca["pop"] = np.concatenate((ca["pop"], belief["pop"]))

In [21]:
def update_beliefs(ca):
    for belief in ca["cultures"]:
        update_traits(belief)

In [22]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
data = data.drop(["ID", "ZIP Code"], axis = 1)

In [23]:
X = data.drop("Personal Loan", axis = 1).values
y = data["Personal Loan"].values.reshape(-1,1)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [25]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [26]:
model = create_network([11, 4, 1], [nn.Sigmoid, nn.Sigmoid])
forwardpass_network(model, X_train)

epochs = 100

ca = create_ca(model, X_train, y_train, [(20, 20),(10,10),(100, 10)], 1)
generate_population(ca, 10)

acc = 0
for epoch in range(epochs):    
    reproduce(ca)
    select_next_population(ca)
    classify(ca)
    update_beliefs(ca)
    evaluate_population(ca)
    acc = max(ca["scores"])
    print("epoch : ",epoch," : ",acc)

epoch :  0  :  0.803
epoch :  1  :  0.9045
epoch :  2  :  0.9045
epoch :  3  :  0.9045
epoch :  4  :  0.9045
epoch :  5  :  0.9045
epoch :  6  :  0.9045
epoch :  7  :  0.9045
epoch :  8  :  0.9045


C:\Users\Achu\AppData\Local\Temp\ipykernel_25204\924963958.py:3: RuntimeWarning: divide by zero encountered in true_divide
  z = np.sum((ca["pop"] - belief["mean"]), axis = 1) / (belief["std"])


epoch :  9  :  0.9045


C:\Users\Achu\AppData\Local\Temp\ipykernel_25204\924963958.py:3: RuntimeWarning: divide by zero encountered in true_divide
  z = np.sum((ca["pop"] - belief["mean"]), axis = 1) / (belief["std"])


epoch :  10  :  0.9045


...

C:\Users\Achu\AppData\Local\Temp\ipykernel_25204\924963958.py:3: RuntimeWarning: divide by zero encountered in true_divide
  z = np.sum((ca["pop"] - belief["mean"]), axis = 1) / (belief["std"])


epoch :  31  :  0.9045
epoch :  32  :  0.9045
epoch :  33  :  0.9045
epoch :  34  :  0.9045
epoch :  35  :  0.9045
epoch :  36  :  0.9045
epoch :  37  :  0.9045
epoch :  38  :  0.9045
epoch :  39  :  0.9045
epoch :  40  :  0.9045
epoch :  41  :  0.9045
epoch :  42  :  0.9045
epoch :  43  :  0.9045
epoch :  44  :  0.9045
epoch :  45  :  0.9045
epoch :  46  :  0.9045


C:\Users\Achu\AppData\Local\Temp\ipykernel_25204\924963958.py:3: RuntimeWarning: divide by zero encountered in true_divide
  z = np.sum((ca["pop"] - belief["mean"]), axis = 1) / (belief["std"])


epoch :  47  :  0.9045


KeyboardInterrupt: 